In [1]:
import os

In [2]:
%pwd

'c:\\Users\\gopal\\End-to-end-Machine-Learning-with-MLOps-\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\gopal\\End-to-end-Machine-Learning-with-MLOps-'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/gopalpranay53-cloud/End-to-end-Machine-Learning-with-MLOps-.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="gopalpranay53-cloud"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d80792694413aac4f517c6366f7d582e500fce18"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from MLOps_Project.constants import *
from MLOps_Project.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/gopalpranay53-cloud/End-to-end-Machine-Learning-with-MLOps-.mlflow",
           
        )

        return model_evaluation_config


In [9]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

C:\Users\gopal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\mlflow\utils\requirements_utils.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources  # noqa: TID251


In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        mlflow.set_experiment("Model_Evaluation")
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    


In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2026-01-21 14:00:55,538: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-21 14:00:55,540: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-21 14:00:55,540: INFO: common: yaml file: schema.yaml loaded successfully]
[2026-01-21 14:00:55,546: INFO: common: created directory at: artifacts]
[2026-01-21 14:00:55,549: INFO: common: created directory at: artifacts/model_evaluation]


2026/01/21 14:00:55 INFO mlflow.tracking.fluent: Experiment with name 'Model_Evaluation' does not exist. Creating a new experiment.


[2026-01-21 14:00:57,070: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2026/01/21 14:01:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 5
Created version '5' of model 'ElasticnetModel'.


In [15]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: https://dagshub.com/gopalpranay53-cloud/End-to-end-Machine-Learning-with-MLOps-.mlflow
